In [13]:
# Import libraries
import pandas as pd
import requests
from pprint import pprint

In [ ]:
# URLs
cntry_url = 'https://restcountries.eu/rest/v2/all'

In [21]:
# Get Countries, Lat, Long
cntry_response = requests.get(cntry_url).json()

# Process each country and store details in a list
cntry_ls = []
for index, each in enumerate(cntry_response):
    try:
        c_name = each['name']
        c_code = each['alpha3Code']
        c_lat = each['latlng'][0]
        c_lng = each['latlng'][1]
        c_pop = each['population']
        c_area = each['area']
        c_details = [c_name, c_code, c_pop, c_area, c_lat, c_lng]
        cntry_ls.append(c_details)
    except:
        print(f"Data not found for {c_name}")

# Convert list to dataframe        
cntry_df = pd.DataFrame(cntry_ls, columns=["Name", "Code", "Population", "Area", "Lat", "Lng"])

# Convert data to numeric
cntry_df["Population"] = pd.to_numeric(cntry_df["Population"])
cntry_df["Area"] = pd.to_numeric(cntry_df["Area"])
cntry_df["Lat"] = pd.to_numeric(cntry_df["Lat"])
cntry_df["Lng"] = pd.to_numeric(cntry_df["Lng"])

# Cross Join year
years = []
for i in range(2000, 2016, 1):
    years.append(i)
years_df = pd.DataFrame(years, columns=["Year"])
years_df

cntry_df['dummykey'] = 0
years_df['dummykey'] = 0

countries_df = cntry_df.merge(years_df, how='outer')

countries_df = countries_df.drop(columns=["dummykey"])

Data not found for United States Minor Outlying Islands


In [18]:
# Get Emission data
emm_df = pd.read_csv("EmissionData.csv", skiprows=4)

# Remove unwanted columns
emm_df = emm_df.drop(columns=["Indicator Name", "Indicator Code"])
emm_df = emm_df.drop(columns=["1960", "1961", "1962", "1963", "1964", "1965", "1966", "1967", "1968", "1969"])
emm_df = emm_df.drop(columns=["1970", "1971", "1972", "1973", "1974", "1975", "1976", "1977", "1978", "1979"])
emm_df = emm_df.drop(columns=["1980", "1981", "1982", "1983", "1984", "1985", "1986", "1987", "1988", "1989"])
emm_df = emm_df.drop(columns=["1990", "1991", "1992", "1993", "1994", "1995", "1996", "1997", "1998", "1999"])
emm_df = emm_df.drop(columns=["2016", "2017", "2018", "Unnamed: 63"])


# Unpivot: Change year columns to rows
emm_df = pd.melt(emm_df, id_vars=['Country Name'], value_vars=['2000', '2001', '2002', '2003', '2004', '2005',
                                                       '2006', '2007', '2008', '2009', '2010', '2011',
                                                       '2012', '2013', '2014', '2015'])
emm_df.columns=["Name", "Year", "Emission"]
emm_df = emm_df.dropna()
emm_df["Year"] = pd.to_numeric(emm_df["Year"])

# Create results with countries and emission data
result = countries_df.merge(emm_df, how='left', on=['Name','Year'])
result.head()

,Name,Code,Population,Area,Lat,Lng,Year,Emission
0,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2000,0.038506
1,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2001,0.039002
2,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2002,0.048716
3,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2003,0.051830
4,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2004,0.039378


In [20]:
# Get Education data
edu = pd.read_csv("Education.csv")
edu.columns=["Name", "Code", "Year", "Education"]
edu = edu.drop(columns=["Code"])

df=result.merge(edu, how="left", on=['Name', 'Year'])
df.head()


,Name,Code,Population,Area,Lat,Lng,Year,Emission,Education
0,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2000,0.038506,NaN
1,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2001,0.039002,NaN
2,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2002,0.048716,NaN
3,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2003,0.051830,1.25684
4,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2004,0.039378,1.25603
